In [1]:
import sys
!{sys.executable} -m pip install pyspark pandas networkx scipy torch texttable

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
import os
os.environ["HADOOP_USER_NAME"] = "xxx"

In [3]:
import subprocess

start_date = "2023-06-03"
end_date = "2023-06-30"
path = "/root/target_dir/JobRec-maintarget_dir/JobRec-main/job"
city_short = "bj"

command = [
    "spark-submit",
    "--master", "local[*]",
    "/root/target_dir/JobRec-main",
    start_date,
    end_date,
    path,
    city_short
]

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(e.stderr)

25/03/24 17:35:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Exception in thread "main" org.apache.spark.SparkException: Failed to get main class in JAR with error '/root/target_dir/JobRec-main (Is a directory)'.  Please specify one with --class.
	at org.apache.spark.deploy.SparkSubmit.error(SparkSubmit.scala:1052)
	at org.apache.spark.deploy.SparkSubmit.prepareSubmitEnvironment(SparkSubmit.scala:533)
	at org.apache.spark.deploy.SparkSubmit.org$apache$spark$deploy$SparkSubmit$$runMain(SparkSubmit.scala:969)
	at org.apache.spark.deploy.SparkSubmit.doRunMain$1(SparkSubmit.scala:199)
	at org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:222)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:91)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1125)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1134)
	at org.apache.spark.deploy.SparkSubmi

In [4]:
import subprocess

try:
    result = subprocess.run(['apt-get', 'update'], capture_output=True, text=True, check=True)
    result = subprocess.run(['apt-get', 'install', '-y', 'openjdk-8-jdk'], capture_output=True, text=True, check=True)
    print("Java安装成功")
except subprocess.CalledProcessError as e:
    print(f"Java安装失败: {e.stderr}")

Java安装成功


In [5]:
import os

java_path = "/usr/lib/jvm/java-8-openjdk-amd64/jre"  # 请根据实际输出修改
os.environ["JAVA_HOME"] = java_path
os.environ["PATH"] = f"{java_path}/bin:{os.environ['PATH']}"

In [6]:
import sys
import os
import time
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import json
import datetime 

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 1.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"

K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]

def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d')
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d')
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

spark = SparkSession\
    .builder\
    .appName("spark_data_query")\
    .config("spark.sql.shuffle.partitions",500)\
    .config("spark.driver.memory", "16g")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

occ = spark.read.parquet(f'{file_path}/sparksteps/0data/202code2_query_{city_short}/occupation')
occ_train = occ.filter(F.col('date').isin(dates_train)).cache()
occ_test = occ.filter(F.col('date').isin(dates_test)).cache()
occ.show(1)
occ.printSchema()
print('occ_train cnt:', occ_train.count())
print('occ_test cnt:', occ_test.count())

# 查看job postion 数量
# job_pos = occ.select(F.col('position_lv3_code')).distinct()
job_pos = occ.filter((F.col('position_lv3_code') >= POS_CODE_LOW) & (F.col('position_lv3_code') < POS_CODE_HIGH))\
                .select(F.col('position_lv3_code')).distinct().collect()
job_pos_list = list(set([row['position_lv3_code'] for row in job_pos]))
print('job pos cnt:', len(job_pos_list))
print(job_pos_list)

print('job 数值化1')
# For training data
# occ_train = occ_train.filter(F.col('position_lv3_code').isin(job_pos_list)).select(\
occ_train = occ_train.select(\
                 F.col('job_id').cast(LongType()),\
                 F.col('city_code').alias('city_code_str'), \
                 F.col('city_name'),\
                 F.col('update_time').cast(LongType()),\
                 F.col('position_lv3_code').alias('position_code').cast(IntegerType()),\
                 F.col('low_salary').cast(IntegerType()),\
                 F.col('high_salary').cast(IntegerType()),\
                 F.col('degree_code').cast(IntegerType()),\
                 F.col('experience_code').cast(IntegerType()),\
                 F.col('title').alias('name'),\
                 F.col('days_per_week_u').cast(IntegerType()),\
                 F.col('area_city_code_u').cast(IntegerType()),\
                 F.to_date(F.col('date')).alias('date'),\
                )
print('\tNULL field process...')
occ_train = occ_train.fillna({'days_per_week_u': 5}).fillna(0).fillna("无")
occ_train.show(1)

# For testing data
# occ_test = occ_test.filter(F.col('position_lv3_code').isin(job_pos_list)).select(\
occ_test = occ_test.select(\
                 F.col('job_id').cast(LongType()),\
                 F.col('city_code').alias('city_code_str'), \
                 F.col('city_name'),\
                 F.col('update_time').cast(LongType()),\
                 F.col('position_lv3_code').alias('position_code').cast(IntegerType()),\
                 F.col('low_salary').cast(IntegerType()),\
                 F.col('high_salary').cast(IntegerType()),\
                 F.col('degree_code').cast(IntegerType()),\
                 F.col('experience_code').cast(IntegerType()),\
                 F.col('title').alias('name'),\
                 F.col('days_per_week_u').cast(IntegerType()),\
                 F.col('area_city_code_u').cast(IntegerType()),\
                 F.to_date(F.col('date')).alias('date'),\
                )
occ_test = occ_test.fillna({'days_per_week_u': 5}).fillna(0).fillna("无")
occ_test.show(1)

print('job 数值化2')
from pyspark.ml.feature import StringIndexer, StringIndexerModel

inputs1 = ["name", "city_code_str"]
outputs1 = ["name_code", "city_code"]

# For training data
stringIndexer1 = StringIndexer(inputCols=inputs1,outputCols=outputs1).setHandleInvalid('keep')
model1 = stringIndexer1.fit(occ_train)
model1.write().overwrite().save(f'{file_path}/sparksteps/models/StringIndexer_occ')

occ_train = model1.transform(occ_train)
# for name in inputs1:
#     occ_train = occ_train.drop(name)
occ_train = occ_train.withColumn('area_city_code_u', F.col('city_code')*10000 + F.col('area_city_code_u'))
occ_train.show(1)

# For testing data
if os.path.exists(f'{file_path}/sparksteps/models/StringIndexer_occ'):
    model1 = StringIndexerModel.load(f'{file_path}/sparksteps/models/StringIndexer_occ')

occ_test = model1.transform(occ_test)
# for name in inputs1:
#     occ_test = occ_test.drop(name)
occ_test = occ_test.withColumn('area_city_code_u', F.col('city_code')*10000 + F.col('area_city_code_u'))
occ_test.show(1)

print('job 数值化3')
def experience_change_to_year(num):
    if num <= 100:
        return 0
    elif num == 101:
        return 0
    elif num == 103:
        return 1
    elif num == 104:
        return 1.5
    elif num == 105:
        return 3
    elif num == 106:
        return 5.5
    elif num == 107:
        return 10
    else:
        return 0 # 应届生/在校生
change_udf1 = F.udf(experience_change_to_year, IntegerType())

def degree_change_to_level(num):
    if num ==209:
        return 1
    elif num == 208:
        return 2
    elif num == 206:
        return 3
    elif num == 202:
        return 4
    elif num == 203:
        return 5
    elif num == 204:
        return 6
    elif num == 205:
        return 7
    else:
        return 0
change_udf2 = F.udf(degree_change_to_level, IntegerType())

# For training data
occ_train = occ_train.withColumn("experience_code", change_udf1(occ_train.experience_code))\
            .withColumn("degree_code", change_udf2(occ_train.degree_code))\
            .drop('city_code')
occ_train.show(1)
occ_train.write.mode('overwrite').parquet(f'{file_path}/sparksteps/train/occ_tmp')
occ_train = occ_train.withColumn("position_code", occ_train.position_code-100000)
for name in inputs1:
    occ_train = occ_train.drop(name)

# For testing data
occ_test = occ_test.withColumn("experience_code", change_udf1(occ_test.experience_code))\
            .withColumn("degree_code", change_udf2(occ_test.degree_code))\
            .drop('city_code')
occ_test.show(1)
occ_test.write.mode('overwrite').parquet(f'{file_path}/sparksteps/test/occ_tmp')
occ_test = occ_test.withColumn("position_code", occ_test.position_code-100000)
for name in inputs1:
    occ_test = occ_test.drop(name)
    
occ_train.show(1)
occ_train = occ_train.dropna(how='any')
occ_test.show(1)
occ_test = occ_test.dropna(how='any')

from pyspark.ml.feature import VectorAssembler


vectorAssembler = VectorAssembler(inputCols=['position_code', 'low_salary', 'high_salary', 'degree_code', \
                                             'experience_code', 'days_per_week_u', 'area_city_code_u', 'name_code'], \
                                  outputCol="features")
# tansdata = vectorAssembler.transform(occ).select('job_id', 'features', 'date')
# tansdata.show(1)

# For training data
tansdata_train = vectorAssembler.transform(occ_train).select('job_id', 'features', 'date')
tansdata_train.show(1)

# For testing data
tansdata_test = vectorAssembler.transform(occ_test).select('job_id', 'features', 'date')
tansdata_test.show(1)



print('数据归一化')
from pyspark.ml.feature import StandardScaler, StandardScalerModel

standardScaler = StandardScaler(inputCol="features",outputCol="scaledFeatures")
print(1)
# For training data
model_one = standardScaler.fit(tansdata_train)
model_one.write().overwrite().save(f'{file_path}/sparksteps/models/StandardScaler_occ')

print(2)
stddata_train = model_one.transform(tansdata_train)
stddata_train.show(1)

# For testing data
if os.path.exists(f'{path}/hanxiao/models/StandardScaler_occ'):
    model_one = StandardScalerModel.load(f'{file_path}/sparksteps/models/StandardScaler_occ')

stddata_test = model_one.transform(tansdata_test)
stddata_test.show(1)



# import sklearn.cluster as cluster
from pyspark.ml.clustering import KMeans, KMeansModel


# # traindata,testdata = stddata.randomSplit([0.8,0.2])
# traindata = stddata.filter(F.col('date').isin(dates_train))
# testdata = stddata.filter(F.col('date').isin(dates_test))
# print(traindata.count(),testdata.count())

print('KMeans')
kmeans = KMeans(featuresCol="scaledFeatures",k=K_job,seed=1)

# For training data
model_two = kmeans.fit(stddata_train) # traindata<->stddata
model_two.write().overwrite().save(f'{file_path}/sparksteps/models/KMeans_occ')
predictions_train = model_two.transform(stddata_train) # testdata<->stddata

# For testing data
if os.path.exists(f'{file_path}/sparksteps/models/KMeans_occ'):
    model_two = KMeansModel.load(f'{file_path}/sparksteps/models/KMeans_occ')

predictions_test = model_two.transform(stddata_test) # testdata<->stddata

from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator

if os.path.exists(f'{file_path}/sparksteps/models/KMeans_occ'):
    model_two = KMeansModel.load(f'{file_path}/sparksteps/models/KMeans_occ')
    
# # Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions_test)
print("Silhouette with squared euclidean distance = " + str(silhouette))

print('Cluster Center')
print(model_two.clusterCenters()[0])
clustercenters = model_two.clusterCenters()
print(len(clustercenters))
print(type(clustercenters))
# print(clustercenters)

import pickle as pkl
with open(f'{path}/sparksteps//models/Clustercenters_occ.pkl', 'wb') as f:
    pkl.dump(clustercenters, f)
    
def extract_feature(vec, index):
    try:
        return float(vec[index])
    except IndexError:
        return None
split_udf = F.udf(extract_feature, FloatType())

# For training data
occ_pred_train = predictions_train.select(F.col('job_id'),\
                                       F.col('date'),\
                                       F.col('prediction'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(0)).alias('position_code'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(1)).alias('low_salary'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(2)).alias('high_salary'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(3)).alias('degree_code'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(4)).alias('exp_year'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(5)).alias('workdays'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(6)).alias('area_code'),\
                                       split_udf(predictions_train.scaledFeatures, F.lit(7)).alias('name_code'))
occ_pred_train.show(20)

# For testing data
occ_pred_test = predictions_test.select(F.col('job_id'),\
                                       F.col('date'),\
                                       F.col('prediction'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(0)).alias('position_code'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(1)).alias('low_salary'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(2)).alias('high_salary'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(3)).alias('degree_code'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(4)).alias('exp_year'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(5)).alias('workdays'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(6)).alias('area_code'),\
                                       split_udf(predictions_test.scaledFeatures, F.lit(7)).alias('name_code'))
occ_pred_test.show(20)

# For training data
occ_pred_train.write.mode('overwrite').parquet(f'{file_path}/sparksteps/train/occ')

# For testing data
occ_pred_test.write.mode('overwrite').parquet(f'{file_path}/sparksteps/test/occ')


dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 17:35:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+---------+-----------+---------+---------+-----------------+-----------------+----------+-----------+-----------+---------------+------------------+---------------+----------------+---------------+----------+
|   job_id|update_time|city_name|city_code|position_lv3_code|position_lv3_name|low_salary|high_salary|degree_code|experience_code|             title|days_per_week_u|area_city_code_u|area_district_u|      date|
+---------+-----------+---------+---------+-----------------+-----------------+----------+-----------+-----------+---------------+------------------+---------------+----------------+---------------+----------+
|288845965| 1685614345|     上海|101020100|           180807|       量化研究员|       400|        700|        203|            109|高频因子挖掘实习生|              4|             021|         长宁区|2023-06-12|
+---------+-----------+---------+---------+-----------------+-----------------+----------+-----------+-----------+---------------+------------------+---------------+--------------

occ_train cnt: 1514796
occ_test cnt: 370088


job pos cnt: 126
['100506', '100403', '101407', '100404', '100804', '100124', '100817', '101308', '101011', '101017', '100706', '101100', '100301', '100210', '101404', '101009', '100701', '100311', '100807', '101301', '101299', '101006', '100116', '101001', '101002', '100601', '101003', '100209', '100801', '100904', '101302', '101309', '100816', '100121', '100805', '101018', '100202', '100405', '100305', '101202', '100606', '101312', '100704', '100205', '100410', '101306', '100302', '100104', '100212', '101008', '101013', '101016', '100105', '100107', '101015', '100806', '100310', '101010', '101406', '101310', '100811', '100199', '100406', '101307', '101007', '100303', '101311', '100122', '100407', '100307', '100109', '101101', '100106', '100401', '101201', '100118', '100705', '101014', '100306', '100703', '100803', '100208', '100304', '100511', '100808', '100512', '100810', '100103', '100114', '100117', '101402', '100402', '100101', '100408', '100308', '101005', '100203', '100409', '1

...

+---------+---------+-----------+-------------+----------+-----------+-----------+---------------+---------------+----------------+----------+---------+
|   job_id|city_name|update_time|position_code|low_salary|high_salary|degree_code|experience_code|days_per_week_u|area_city_code_u|      date|name_code|
+---------+---------+-----------+-------------+----------+-----------+-----------+---------------+---------------+----------------+----------+---------+
|288845965|     上海| 1685614345|        80807|       400|        700|          5|              0|              4|         10021.0|2023-06-12|  82672.0|
+---------+---------+-----------+-------------+----------+-----------+-----------+---------------+---------------+----------------+----------+---------+
only showing top 1 row

+---------+---------+-----------+-------------+----------+-----------+-----------+---------------+---------------+----------------+----------+---------+
|   job_id|city_name|update_time|position_code|low_salary|hi

2
+---------+--------------------+----------+--------------------+
|   job_id|            features|      date|      scaledFeatures|
+---------+--------------------+----------+--------------------+
|288845965|[80807.0,400.0,70...|2023-06-12|[1.72827604337581...|
+---------+--------------------+----------+--------------------+
only showing top 1 row

+---------+--------------------+----------+--------------------+
|   job_id|            features|      date|      scaledFeatures|
+---------+--------------------+----------+--------------------+
|286718149|[80118.0,15.0,18....|2023-06-27|[1.71353991663077...|
+---------+--------------------+----------+--------------------+
only showing top 1 row

KMeans


Silhouette with squared euclidean distance = -0.17320950653382725
Cluster Center
[1.66373167e-02 4.80750880e-02 5.76854434e-02 2.94378618e+00
 0.00000000e+00 3.33838669e+01 7.05860601e-03 1.84688550e-02]
500
<class 'list'>


+---------+----------+----------+-------------+-----------+-----------+-----------+---------+---------+------------+-------------+
|   job_id|      date|prediction|position_code| low_salary|high_salary|degree_code| exp_year| workdays|   area_code|    name_code|
+---------+----------+----------+-------------+-----------+-----------+-----------+---------+---------+------------+-------------+
|288845965|2023-06-12|       276|     1.728276|  1.1055187|  1.3568841|  2.9437861|      0.0|26.707094|  0.08355995|    1.6338236|
|288862726|2023-06-12|       226|    4.0684543| 0.01658278|0.013568841|        0.0|      0.0|33.383865|8.3384846E-5|     3.454741|
|283438444|2023-06-12|         2|  0.008662019|0.030401763| 0.04264493|  2.9437861|1.4627733|33.383865|         0.0| 0.0043477984|
|280258590|2023-06-12|       115|  0.002160158|0.022110373|0.023260871|  2.9437861|1.4627733|33.383865|   0.8367753|1.18576325E-4|
|289381991|2023-06-12|       421|   0.21603718| 0.04974834|0.054275364|  2.9437861|

+---------+----------+----------+-------------+-----------+-----------+-----------+---------+---------+------------+------------+
|   job_id|      date|prediction|position_code| low_salary|high_salary|degree_code| exp_year| workdays|   area_code|   name_code|
+---------+----------+----------+-------------+-----------+-----------+-----------+---------+---------+------------+------------+
|286718149|2023-06-27|       329|      1.71354| 0.04145695|0.034891304|  3.5325434|      0.0|33.383865|8.3384846E-5|   0.8295007|
|283904464|2023-06-27|       108|  0.012854009| 0.03316556|0.029076088|  2.9437861|      0.0|33.383865|  0.41951752|5.9288162E-5|
|242540699|2023-06-27|         5|  0.006630188| 0.01658278|0.019384058|        0.0|      0.0|33.383865|8.3384846E-5|  0.06079013|
|276671604|2023-06-27|       177|    4.2905016| 0.01658278|0.013568841|  2.3550289|1.4627733|33.383865|8.3384846E-5| 0.041284323|
|262810834|2023-06-27|        71|   0.01927032|0.027637966|0.021322465|  2.3550289|1.46277

In [7]:
import sys
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import json
import datetime 
import time

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 2.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"

K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]



def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

spark = SparkSession\
    .builder\
    .appName("spark_data_query")\
    .config("spark.sql.shuffle.partitions",500)\
    .config("spark.driver.memory", "32g")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

person = spark.read.parquet(f'{file_path}/sparksteps/0data/202code2_query_{city_short}/person')
person_train = person.filter(F.col('date').isin(dates_train)).cache()
person_test = person.filter(F.col('date').isin(dates_test)).cache()
person.show(1)
person.printSchema()
print('person cnt:', person.count())
print('person_train cnt:', person_train.count())
print('person_test cnt:', person_test.count())

print('job 数值化1')

# For training data
person_train = person_train.select(\
                 F.col('geek_id').cast(LongType()),\
                 F.col('age').cast(IntegerType()), \
                 F.col('gender').cast(IntegerType()), \
                 F.col('degree_code').cast(IntegerType()),\
                 F.col('work_years').cast(IntegerType()),\
                 F.col('fresh_graduate').cast(IntegerType()),\
                 F.col('apply_status').cast(IntegerType()),\
                 F.col('expect1_update_time').cast(LongType()),\
                 F.col('expect2_update_time').cast(LongType()),\
                 F.col('expect3_update_time').cast(LongType()),\
                 F.col('edu1_education_id').alias('edu1_id').cast(IntegerType()),\
                 F.col('edu1_degree_code').cast(IntegerType()),\
                 F.col('edu1_standard_major_name').alias('edu1_major'),\
                 F.col('edu1_start_date').cast(LongType()),\
                 F.col('edu1_end_date').cast(LongType()),\
                 F.col('edu2_education_id').alias('edu2_id').cast(IntegerType()),\
                 F.col('edu2_degree_code').cast(IntegerType()),\
                 F.col('edu2_standard_major_name').alias('edu2_major'),\
                 F.col('edu2_start_date').cast(LongType()),\
                 F.col('edu2_end_date').cast(LongType()),\
                 F.col('edu3_education_id').alias('edu3_id').cast(IntegerType()),\
                 F.col('edu3_degree_code').cast(IntegerType()),\
                 F.col('edu3_standard_major_name').alias('edu3_major'),\
                 F.col('edu3_start_date').cast(LongType()),\
                 F.col('edu3_end_date').cast(LongType()),\
                 F.col('work1_position_lv3_code').cast(IntegerType()),\
                 F.col('work1_industry_lv1_code').cast(IntegerType()),\
                 F.col('work1_skills'),\
                 F.col('work1_start_date').cast(LongType()),\
                 F.col('work1_end_date').cast(LongType()),\
                 F.col('work2_position_lv3_code').cast(IntegerType()),\
                 F.col('work2_industry_lv1_code').cast(IntegerType()),\
                 F.col('work2_skills'),\
                 F.col('work2_start_date').cast(LongType()),\
                 F.col('work2_end_date').cast(LongType()),\
                 F.col('work3_position_lv3_code').cast(IntegerType()),\
                 F.col('work3_industry_lv1_code').cast(IntegerType()),\
                 F.col('work3_skills'),\
                 F.col('work3_start_date').cast(LongType()),\
                 F.col('work3_end_date').cast(LongType()),\
                 F.to_date(F.col('date')).alias('date'),\
                )
print('\tNULL field process...')
person_train = person_train.fillna({'work1_industry_lv1_code': 80000,\
                       'work2_industry_lv1_code': 80000,\
                       'work3_industry_lv1_code': 80000,\
                       }).fillna(0).fillna("无")
person_train.show(1)



# For testing data
person_test = person_test.select(\
                 F.col('geek_id').cast(LongType()),\
                 F.col('age').cast(IntegerType()), \
                 F.col('gender').cast(IntegerType()), \
                 F.col('degree_code').cast(IntegerType()),\
                 F.col('work_years').cast(IntegerType()),\
                 F.col('fresh_graduate').cast(IntegerType()),\
                 F.col('apply_status').cast(IntegerType()),\
                 F.col('expect1_update_time').cast(LongType()),\
                 F.col('expect2_update_time').cast(LongType()),\
                 F.col('expect3_update_time').cast(LongType()),\
                 F.col('edu1_education_id').alias('edu1_id').cast(IntegerType()),\
                 F.col('edu1_degree_code').cast(IntegerType()),\
                 F.col('edu1_standard_major_name').alias('edu1_major'),\
                 F.col('edu1_start_date').cast(LongType()),\
                 F.col('edu1_end_date').cast(LongType()),\
                 F.col('edu2_education_id').alias('edu2_id').cast(IntegerType()),\
                 F.col('edu2_degree_code').cast(IntegerType()),\
                 F.col('edu2_standard_major_name').alias('edu2_major'),\
                 F.col('edu2_start_date').cast(LongType()),\
                 F.col('edu2_end_date').cast(LongType()),\
                 F.col('edu3_education_id').alias('edu3_id').cast(IntegerType()),\
                 F.col('edu3_degree_code').cast(IntegerType()),\
                 F.col('edu3_standard_major_name').alias('edu3_major'),\
                 F.col('edu3_start_date').cast(LongType()),\
                 F.col('edu3_end_date').cast(LongType()),\
                 F.col('work1_position_lv3_code').cast(IntegerType()),\
                 F.col('work1_industry_lv1_code').cast(IntegerType()),\
                 F.col('work1_skills'),\
                 F.col('work1_start_date').cast(LongType()),\
                 F.col('work1_end_date').cast(LongType()),\
                 F.col('work2_position_lv3_code').cast(IntegerType()),\
                 F.col('work2_industry_lv1_code').cast(IntegerType()),\
                 F.col('work2_skills'),\
                 F.col('work2_start_date').cast(LongType()),\
                 F.col('work2_end_date').cast(LongType()),\
                 F.col('work3_position_lv3_code').cast(IntegerType()),\
                 F.col('work3_industry_lv1_code').cast(IntegerType()),\
                 F.col('work3_skills'),\
                 F.col('work3_start_date').cast(LongType()),\
                 F.col('work3_end_date').cast(LongType()),\
                 F.to_date(F.col('date')).alias('date'),\
                )
print('\tNULL field process...')
person_test = person_test.fillna({'work1_industry_lv1_code': 80000,\
                       'work2_industry_lv1_code': 80000,\
                       'work3_industry_lv1_code': 80000,\
                       }).fillna(0).fillna("无")
person_test.show(1)

print('job 数值化2')
def degree_change_to_level(num):
    if num ==209:
        return 1
    elif num == 208:
        return 2
    elif num == 206:
        return 3
    elif num == 202:
        return 4
    elif num == 203:
        return 5
    elif num == 204:
        return 6
    elif num == 205:
        return 7
    else:
        return 0
change_udf = F.udf(degree_change_to_level, IntegerType())

# For training data
person_train = person_train.withColumn("degree_code", change_udf(person_train.degree_code))\
            .withColumn("edu1_degree_code", change_udf(person_train.edu1_degree_code))\
            .withColumn("edu2_degree_code", change_udf(person_train.edu2_degree_code))\
            .withColumn("edu3_degree_code", change_udf(person_train.edu3_degree_code))\
            .withColumn("work1_position_lv3_code", person_train.work1_position_lv3_code-100000)\
            .withColumn("work2_position_lv3_code", person_train.work2_position_lv3_code-100000)\
            .withColumn("work3_position_lv3_code", person_train.work3_position_lv3_code-100000)\
            .withColumn("work2_industry_lv1_code", person_train.work2_industry_lv1_code-100000)
person_train.show(1)
person_train.write.mode('overwrite').parquet(f'{file_path}/sparksteps/train/person_tmp')

# For testing data
person_test = person_test.withColumn("degree_code", change_udf(person_test.degree_code))\
            .withColumn("edu1_degree_code", change_udf(person_test.edu1_degree_code))\
            .withColumn("edu2_degree_code", change_udf(person_test.edu2_degree_code))\
            .withColumn("edu3_degree_code", change_udf(person_test.edu3_degree_code))\
            .withColumn("work1_position_lv3_code", person_test.work1_position_lv3_code-100000)\
            .withColumn("work2_position_lv3_code", person_test.work2_position_lv3_code-100000)\
            .withColumn("work3_position_lv3_code", person_test.work3_position_lv3_code-100000)\
            .withColumn("work2_industry_lv1_code", person_test.work2_industry_lv1_code-100000)
person_test.show(1)
person_test.write.mode('overwrite').parquet(f'{file_path}/sparksteps/test/person_tmp')


def concat_arrays(array1, array2, array3, array4):
    return array1 + array2 + array3 + array4


concat_udf = F.udf(concat_arrays, ArrayType(StringType()))

# For training data
person_train = person_train.withColumn("text_combined", concat_udf(F.array("edu1_major", "edu2_major", "edu3_major"), \
                                          F.split("work1_skills", "、"), F.split("work2_skills", "、"), F.split("work3_skills", "、")))\
        .drop('edu1_major', 'edu2_major', 'edu3_major', 'work1_skills', 'work2_skills', 'work3_skills')
person_train.show(1)

# For testing data
person_test = person_test.withColumn("text_combined", concat_udf(F.array("edu1_major", "edu2_major", "edu3_major"), \
                                          F.split("work1_skills", "、"), F.split("work2_skills", "、"), F.split("work3_skills", "、")))\
        .drop('edu1_major', 'edu2_major', 'edu3_major', 'work1_skills', 'work2_skills', 'work3_skills')
person_test.show(1)

from pyspark.ml.feature import CountVectorizer, CountVectorizerModel
from pyspark.ml.clustering import LDA, LocalLDAModel

# For training data
data_selected_train = person_train.select("geek_id", "date", "text_combined").cache()
data_selected_train.show(1)

# For testing data
data_selected_test = person_test.select("geek_id", "date", "text_combined").cache()
data_selected_test.show(1)


print('\tCalculating Word Freq')
vectorizer = CountVectorizer(inputCol="text_combined",
                             outputCol="features")

# For training data
model = vectorizer.fit(data_selected_train)
model.write().overwrite().save(f'{file_path}/sparksteps/models/CountVectorizer_person')

featuresData_train = model.transform(data_selected_train)
# featuresData.show(3)

# For testing data
if os.path.exists(f'{path}/models/CountVectorizer_person'):
    model = CountVectorizerModel.load(f'{file_path}/sparksteps/models/CountVectorizer_person')
    
featuresData_test = model.transform(data_selected_test)
# featuresData.show(3)

print('\tLda traininig...')
# For training data
lda = LDA(k=numTopics, maxIter=100)
ldaModel = lda.fit(featuresData_train)
ldaModel.write().overwrite().save(f'{file_path}/sparksteps/models/LDA_person')

# For testing data
if os.path.exists(f'{path}/models/LDA_person'):
    ldaModel = LocalLDAModel.load(f'{file_path}/sparksteps/models/LDA_person')

print('\tGet Topic')
# For training data
transformedData_train = ldaModel.transform(featuresData_train)
# transformedData.show()

# For testing data
transformedData_test = ldaModel.transform(featuresData_test)

print('\tUnderstand Topic')
def getTopTopics(args):
    geekId, date, topicDistribution = args
    topTopics = topicDistribution.toArray().argsort()[-1:][::-1][0]
    return (geekId, date, topTopics.tolist())

print('\tGet topic per person')
# For training data
documentTopics_train = transformedData_train.rdd.map(lambda row: (row["geek_id"], row["date"], row["topicDistribution"])) \
    .map(getTopTopics) \
    .toDF(["geek_id1", "date1", "skills"])
documentTopics_train.show(1, truncate=False)

# For testing data
documentTopics_test = transformedData_test.rdd.map(lambda row: (row["geek_id"], row["date"], row["topicDistribution"])) \
    .map(getTopTopics) \
    .toDF(["geek_id1", "date1", "skills"])
documentTopics_test.show(1, truncate=False)

print('\tjoin operator')
# For training data
person_train = person_train.join(documentTopics_train, (person_train.geek_id == documentTopics_train.geek_id1) & (person_train.date == documentTopics_train.date1), \
                     how="left").drop("text_combined", "geek_id1", "date1").cache()
person_train.show(1)
print('person cnt:', person_train.count())


# For testing data
person_test = person_test.join(documentTopics_test, (person_test.geek_id == documentTopics_test.geek_id1) & (person_test.date == documentTopics_test.date1), \
                     how="left").drop("text_combined", "geek_id1", "date1").cache()
person_test.show(1)
print('person cnt:', person_train.count())

from pyspark.ml.feature import VectorAssembler


vectorAssembler = VectorAssembler(inputCols=['age','gender','degree_code','work_years','fresh_graduate','apply_status',\
                                             'expect1_update_time','expect2_update_time','expect3_update_time','skills',\
                                             'edu1_id','edu1_degree_code','edu1_start_date','edu1_end_date',\
                                             'edu2_id','edu2_degree_code','edu2_start_date','edu2_end_date',\
                                             'edu3_id','edu3_degree_code','edu3_start_date','edu3_end_date',\
                                             'work1_position_lv3_code','work1_industry_lv1_code','work1_start_date','work1_end_date',\
                                             'work2_position_lv3_code','work2_industry_lv1_code','work2_start_date','work2_end_date',\
                                             'work3_position_lv3_code','work3_industry_lv1_code','work3_start_date','work3_end_date'],\
                                  outputCol="features")
# For training data
tansdata_train = vectorAssembler.transform(person_train).select('geek_id', 'features', 'date')
tansdata_train.show(1)

# For testing data
tansdata_test = vectorAssembler.transform(person_test).select('geek_id', 'features', 'date')
tansdata_test.show(1)

print('数据归一化')
from pyspark.ml.feature import StandardScaler, StandardScalerModel

# For training data
standardScaler = StandardScaler(inputCol="features",outputCol="scaledFeatures")
    
model_one = standardScaler.fit(tansdata_train)
model_one.write().overwrite().save(f'{file_path}/sparksteps/models/StandardScalerModel_person')

stddata_train = model_one.transform(tansdata_train)
stddata_train.show(1)

# For testing data
if os.path.exists(f'{path}/models/StandardScalerModel_person'):
    model_one = StandardScalerModel.load(f'{file_path}/sparksteps/models/StandardScalerModel_person')

stddata_test = model_one.transform(tansdata_test)
stddata_test.show(1)

from pyspark.ml.clustering import KMeans, KMeansModel


# # traindata,testdata = stddata.randomSplit([0.8,0.2])
# traindata = stddata.filter(F.col('date').isin(dates_train))
# testdata = stddata.filter(F.col('date').isin(dates_test))
# print(traindata.count(),testdata.count())

print('KMeans')
# For training data
kmeans = KMeans(featuresCol="scaledFeatures",k=K_person,seed=1)

model_two = kmeans.fit(stddata_train) # traindata<->stddata
model_two.write().overwrite().save(f'{file_path}/sparksteps/models/KMeans_person')

predictions_train = model_two.transform(stddata_train) # testdata<->stddata

# For testing data
if os.path.exists(f'{path}/models/KMeans_person'):
    model_two = KMeansModel.load(f'{file_path}/sparksteps/models/KMeans_person')
    
predictions_test = model_two.transform(stddata_test) # testdata<->stddata

from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator

if os.path.exists(f'{path}/models/KMeans_person'):
    model_two = KMeansModel.load(f'{file_path}/sparksteps/models/KMeans_person')

# # Evaluate clustering by computing Silhouette score
# evaluator = ClusteringEvaluator()

# silhouette = evaluator.evaluate(predictions_test)
# print("Silhouette with squared euclidean distance = " + str(silhouette))

print('Cluster Center')
# print(model_two.clusterCenters())
clustercenters = model_two.clusterCenters()
print(len(clustercenters))
print(type(clustercenters))

import pickle as pkl
with open(f'{path}/sparksteps/models/Clustercenters_person.pkl', 'wb') as f:
    pkl.dump(clustercenters, f)
    
def extract_feature(vec, index):
    try:
        return float(vec[index])
    except IndexError:
        return None
split_udf = F.udf(extract_feature, FloatType())

# For training data
person_pred_train = predictions_train.select(F.col('geek_id'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(0)).alias('age'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(1)).alias('gender'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(2)).alias('degree_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(3)).alias('work_years'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(4)).alias('fresh_graduate'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(5)).alias('apply_status'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(6)).alias('expect1_update_time'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(7)).alias('expect2_update_time'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(8)).alias('expect3_update_time'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(9)).alias('skills'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(10)).alias('edu1_id'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(11)).alias('edu1_degree_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(12)).alias('edu1_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(13)).alias('edu1_end_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(14)).alias('edu2_id'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(15)).alias('edu2_degree_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(16)).alias('edu2_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(17)).alias('edu2_end_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(18)).alias('edu3_id'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(19)).alias('edu3_degree_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(20)).alias('edu3_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(21)).alias('edu3_end_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(22)).alias('work1_position_lv3_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(23)).alias('work1_industry_lv1_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(24)).alias('work1_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(25)).alias('work1_end_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(26)).alias('work2_position_lv3_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(27)).alias('work2_industry_lv1_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(28)).alias('work2_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(29)).alias('work2_end_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(30)).alias('work3_position_lv3_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(31)).alias('work3_industry_lv1_code'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(32)).alias('work3_start_date'),\
                                          split_udf(predictions_train.scaledFeatures, F.lit(33)).alias('work3_end_date'),\
                                          F.col('date'),
                                          F.col('prediction'))
person_pred_train.show(1)

# For testing data
person_pred_test = predictions_test.select(F.col('geek_id'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(0)).alias('age'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(1)).alias('gender'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(2)).alias('degree_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(3)).alias('work_years'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(4)).alias('fresh_graduate'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(5)).alias('apply_status'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(6)).alias('expect1_update_time'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(7)).alias('expect2_update_time'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(8)).alias('expect3_update_time'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(9)).alias('skills'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(10)).alias('edu1_id'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(11)).alias('edu1_degree_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(12)).alias('edu1_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(13)).alias('edu1_end_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(14)).alias('edu2_id'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(15)).alias('edu2_degree_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(16)).alias('edu2_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(17)).alias('edu2_end_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(18)).alias('edu3_id'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(19)).alias('edu3_degree_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(20)).alias('edu3_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(21)).alias('edu3_end_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(22)).alias('work1_position_lv3_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(23)).alias('work1_industry_lv1_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(24)).alias('work1_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(25)).alias('work1_end_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(26)).alias('work2_position_lv3_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(27)).alias('work2_industry_lv1_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(28)).alias('work2_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(29)).alias('work2_end_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(30)).alias('work3_position_lv3_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(31)).alias('work3_industry_lv1_code'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(32)).alias('work3_start_date'),\
                                          split_udf(predictions_test.scaledFeatures, F.lit(33)).alias('work3_end_date'),\
                                          F.col('date'),
                                          F.col('prediction'))
person_pred_test.show(1)

person_pred_train.write.partitionBy("prediction").mode('overwrite').parquet(f'{file_path}/sparksteps/train/person')
person_pred_test.write.partitionBy("prediction").mode('overwrite').parquet(f'{file_path}/sparksteps/test/person')

print('person train cnt:', person_pred_train.count())
print('person test cnt:', person_pred_test.count())


dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']
+---------+----+------+-----------+----------+--------------+------------+-------------------+-----------------+-------------------+-----------------+-------------------+-----------------+-----------------+----------------+------------------------+---------------+-------------+-----------------+----------------+------------------------+---------------+-------------+-----------------+----------------+------------------------+---------------+-------------+-----------------------+-----------------------+-----------------------+-----------------------+----------------------------

person cnt: 44642


person_train cnt: 36133
person_test cnt: 8509
job 数值化1
	NULL field process...
+---------+---+------+-----------+----------+--------------+------------+-------------------+-------------------+-------------------+---------+----------------+----------------+---------------+-------------+---------+----------------+------------------+---------------+-------------+-------+----------------+----------+---------------+-------------+-----------------------+-----------------------+-------------------------------------+----------------+--------------+-----------------------+-----------------------+----------------------+----------------+--------------+-----------------------+-----------------------+------------------+----------------+--------------+----------+
|  geek_id|age|gender|degree_code|work_years|fresh_graduate|apply_status|expect1_update_time|expect2_update_time|expect3_update_time|  edu1_id|edu1_degree_code|      edu1_major|edu1_start_date|edu1_end_date|  edu2_id|edu2_degree_code|       

+--------+---+------+-----------+----------+--------------+------------+-------------------+-------------------+-------------------+--------+----------------+--------------------------+---------------+-------------+-------+----------------+------------------+---------------+-------------+-------+----------------+----------+---------------+-------------+-----------------------+-----------------------+------------------------+----------------+--------------+-----------------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+-------------------------------------+----------------+--------------+----------+
| geek_id|age|gender|degree_code|work_years|fresh_graduate|apply_status|expect1_update_time|expect2_update_time|expect3_update_time| edu1_id|edu1_degree_code|                edu1_major|edu1_start_date|edu1_end_date|edu2_id|edu2_degree_code|        edu2_major|edu2_start_date|edu2_end_date|edu3_id|edu3_degr

+---------+---+------+-----------+----------+--------------+------------+-------------------+-------------------+-------------------+---------+----------------+---------------+-------------+---------+----------------+---------------+-------------+-------+----------------+---------------+-------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+----------+----------------------------------+
|  geek_id|age|gender|degree_code|work_years|fresh_graduate|apply_status|expect1_update_time|expect2_update_time|expect3_update_time|  edu1_id|edu1_degree_code|edu1_start_date|edu1_end_date|  edu2_id|edu2_degree_code|edu2_start_date|edu2_end_date|edu3_id|edu3_degree_code|edu3_start_date|edu3_end_date|work1_position_lv3_code|work1_industry_lv1_code|work1_start_date|work1_end_date|work2_position_lv3_code|work2

...

+---------+----------+------+
|geek_id1 |date1     |skills|
+---------+----------+------+
|502467001|2023-06-06|95    |
+---------+----------+------+
only showing top 1 row



+--------+----------+------+
|geek_id1|date1     |skills|
+--------+----------+------+
|28317633|2023-06-29|183   |
+--------+----------+------+
only showing top 1 row

	join operator


+--------+---+------+-----------+----------+--------------+------------+-------------------+-------------------+-------------------+--------+----------------+---------------+-------------+--------+----------------+---------------+-------------+-------+----------------+---------------+-------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+----------+------+
| geek_id|age|gender|degree_code|work_years|fresh_graduate|apply_status|expect1_update_time|expect2_update_time|expect3_update_time| edu1_id|edu1_degree_code|edu1_start_date|edu1_end_date| edu2_id|edu2_degree_code|edu2_start_date|edu2_end_date|edu3_id|edu3_degree_code|edu3_start_date|edu3_end_date|work1_position_lv3_code|work1_industry_lv1_code|work1_start_date|work1_end_date|work2_position_lv3_code|work2_industry_lv1_code|work2_start_dat

+--------+---+------+-----------+----------+--------------+------------+-------------------+-------------------+-------------------+--------+----------------+---------------+-------------+-------+----------------+---------------+-------------+-------+----------------+---------------+-------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+-----------------------+-----------------------+----------------+--------------+----------+------+
| geek_id|age|gender|degree_code|work_years|fresh_graduate|apply_status|expect1_update_time|expect2_update_time|expect3_update_time| edu1_id|edu1_degree_code|edu1_start_date|edu1_end_date|edu2_id|edu2_degree_code|edu2_start_date|edu2_end_date|edu3_id|edu3_degree_code|edu3_start_date|edu3_end_date|work1_position_lv3_code|work1_industry_lv1_code|work1_start_date|work1_end_date|work2_position_lv3_code|work2_industry_lv1_code|work2_start_date|

person train cnt: 36133
person test cnt: 8509


In [8]:
import sys
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import json
import datetime 
import time

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 3.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"

K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]

def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

spark = SparkSession\
    .builder\
    .appName("spark_data_query")\
    .config("spark.sql.shuffle.partitions",500)\
    .config("spark.driver.memory", "32g")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

log = spark.read.parquet(f'{file_path}/sparksteps/0data/202code2_query_{city_short}/log')
log_train = log.filter(F.col('date').isin(dates_train)).cache()
log_test = log.filter(F.col('date').isin(dates_test)).cache()
log.show(20)
log.printSchema()
print('relation cnt:', log.count())
print('log_train cnt:', log_train.count())
print('log_test cnt:', log_test.count())

# For training data
person_train = spark.read.parquet(f'{file_path}/sparksteps/train/person')\
                        .select(F.col('geek_id').alias('geekID'),F.col('prediction').alias('pred_person'),F.col('date').alias('date_person'))
person_train.show(1)
print('person train cnt:', person_train.count())

# For testing data
person_test = spark.read.parquet(f'{file_path}/sparksteps/test/person')\
                        .select(F.col('geek_id').alias('geekID'),F.col('prediction').alias('pred_person'),F.col('date').alias('date_person'))
person_test.show(1)
print('person test cnt:', person_test.count())

# For training data
tmp_person_train = person_train.distinct()
print('tmp person train cnt:', tmp_person_train.count())

# For testing data
tmp_person_test = person_test.distinct()
print('tmp person test cnt:', tmp_person_test.count())

# For training data
occ_train = spark.read.parquet(f'{file_path}/sparksteps/train/occ')\
                    .select(F.col('job_id').alias('jobID'),F.col('prediction').alias('pred_job'),F.col('date').alias('date_job'))
occ_train.show(1)
print('occ train cnt:', occ_train.count())

# For testing data
occ_test = spark.read.parquet(f'{file_path}/sparksteps/test/occ')\
                    .select(F.col('job_id').alias('jobID'),F.col('prediction').alias('pred_job'),F.col('date').alias('date_job'))
occ_test.show(1)
print('occ test cnt:', occ_test.count())

# For training data
tmp_occ_train = occ_train.distinct()
print('tmp occ train cnt:', tmp_occ_train.count())

# For testing data
tmp_occ_test = occ_test.distinct()
print('tmp occ test cnt:', tmp_occ_test.count())

# For training data
log_train = log_train.join(person_train, (log_train.geek_id == person_train.geekID) & (log_train.ds == person_train.date_person), 'left')\
            .drop("geekID", 'date_person').filter(F.col('pred_person').isNotNull())
log_train = log_train.join(occ_train, (log_train.job_id == occ_train.jobID) & (log_train.ds == occ_train.date_job), 'left')\
            .drop('jobID', 'date_job').filter(F.col('pred_job').isNotNull())
log_train.show(1)
print('log train cnt:', log_train.count())

# For testing data
log_test = log_test.join(person_test, (log_test.geek_id == person_test.geekID) & (log_test.ds == person_test.date_person), 'left')\
            .drop("geekID", 'date_person').filter(F.col('pred_person').isNotNull())
log_test = log_test.join(occ_test, (log_test.job_id == occ_test.jobID) & (log_test.ds == occ_test.date_job), 'left')\
            .drop('jobID', 'date_job').filter(F.col('pred_job').isNotNull())
log_test.show(1)
print('log test cnt:', log_test.count())

# For training data
log_train = log_train.orderBy('pred_person', 'pred_job', 'geek_id', 'job_id', 'date')
log_train.show(1)

# For testing data
log_test = log_test.orderBy('pred_person', 'pred_job', 'geek_id', 'job_id', 'date')
log_test.show(1)

# log.orderBy('geek_id', 'deal_time', 'pred_person', 'pred_job', 'job_id').show(10000)

log_train.write.mode('overwrite').parquet(f'{file_path}/sparksteps/train/log')
log_test.write.mode('overwrite').parquet(f'{file_path}/sparksteps/test/log')
print('log train cnt:', log_train.count())
print('log test cnt:', log_test.count())

print('log train cnt:', log_train.distinct().count())
print('log test cnt:', log_test.distinct().count())




dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']
+---------+---------+---------+---------+----+----+----------+----------+----------+----------+
|  geek_id|expect_id|   job_id|deal_type|page|rank| list_time| deal_time|        ds|      date|
+---------+---------+---------+---------+----+----+----------+----------+----------+----------+
|511625017|507990588|264144118|  no_list|   1|  63|1686550101|1686550101|2023-06-12|2023-06-12|
| 20075741|  4114431|282889049|      det|   1|   1|1686536735|1686536739|2023-06-12|2023-06-12|
|557749721|606045715|275914767|  no_list|   3|  31|1686551929|1686551929|2023-06-12|2023-06-12|
|11290

relation cnt: 3802340


log_train cnt: 3038059


log_test cnt: 764281


+--------+-----------+-----------+
|  geekID|pred_person|date_person|
+--------+-----------+-----------+
|28011567|         32| 2023-06-14|
+--------+-----------+-----------+
only showing top 1 row



...

+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
| geek_id|expect_id|   job_id|deal_type|page|rank| list_time| deal_time|        ds|      date|pred_person|pred_job|
+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
|39472752|498715271|236013163|  no_list|   3| 156|1687808106|1687808106|2023-06-27|2023-06-27|          0|       0|
+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
only showing top 1 row



log train cnt: 1478255


log test cnt: 361874


log train cnt: 1474995


log test cnt: 361741


In [9]:
import sys
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import time
import datetime 
import json

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 4.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"


ddays = 7


K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]

def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')


    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

spark = SparkSession\
    .builder\
    .appName("spark_data_query")\
    .config("spark.sql.shuffle.partitions",500)\
    .config("spark.driver.memory", "32g")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# For training data
log_train = spark.read.parquet(f'{file_path}/sparksteps/train/log').cache()
log_train.show(1)
# person.printSchema()
print('log train cnt:', log_train.count())

# For testing data
log_test = spark.read.parquet(f'{file_path}/sparksteps//test/log').cache()
log_test.show(1)
# person.printSchema()
print('log test cnt:', log_test.count())

# For training data



windowSpec = Window.partitionBy("geek_id").orderBy("deal_time")

# log = log.withColumn("next_deal_time", F.lead("deal_time", 1).over(windowSpec))
# log = log.withColumn("next_pred_person", F.lead("pred_person", 1).over(windowSpec))

# time_diff = (F.col("next_deal_time") - F.col("deal_time")) / 86400
# log = log.withColumn("time_diff", time_diff)
# log = log.filter((F.col("pred_person") == F.col("next_pred_person")) & (F.col("time_diff") <= ddays))

log_train = log_train.withColumn("prev_deal_time", F.lag("deal_time").over(windowSpec))
log_train = log_train.withColumn("time_diff", F.when(F.isnull(log_train.deal_time - log_train.prev_deal_time), 0)
                                  .otherwise(log_train.deal_time - log_train.prev_deal_time))

log_train = log_train.withColumn("new_record", F.when(log_train.time_diff > ddays * 86400, 1).otherwise(0))

log_train = log_train.withColumn("group_id", F.sum("new_record").over(windowSpec))

# from pyspark.ml.feature import StringIndexer
# inputs1 = ["job_id"]
# outputs1 = ["job_code"]
# stringIndexer1 = StringIndexer(inputCols=inputs1,outputCols=outputs1).setHandleInvalid('keep')
# model1 = stringIndexer1.fit(log_train)
# model1.write().overwrite().save(f'{file_path}/sparksteps/models/StringIndexer_prepare')
# log_train = model1.transform(log_train)

# min_job_code = log.select(F.min("job_code")).first()[0]
# max_job_code = log.select(F.max("job_code")).first()[0]

# print(f"min new job id：{min_job_code}")
# print(f"max new job id：{max_job_code}")

log_train = log_train.groupBy("geek_id", "group_id").agg(
    F.collect_list("job_id").alias("job_id_list"),
    F.collect_list("deal_type").alias("deal_type_list"),
    F.collect_list("deal_time").alias("deal_time_list"),
    F.collect_list("date").alias("date_list"),
    F.collect_list("pred_person").alias("pred_person_list"),
    F.collect_list("pred_job").alias("pred_job_list")
).drop("group_id").withColumn("list_size", F.size("job_id_list"))
log_train.show()

# For testing data



windowSpec = Window.partitionBy("geek_id").orderBy("deal_time")
log_test = log_test.withColumn("prev_deal_time", F.lag("deal_time").over(windowSpec))
log_test = log_test.withColumn("time_diff", F.when(F.isnull(log_test.deal_time - log_test.prev_deal_time), 0)
                                  .otherwise(log_test.deal_time - log_test.prev_deal_time))

log_test = log_test.withColumn("new_record", F.when(log_test.time_diff > ddays * 86400, 1).otherwise(0))

log_test = log_test.withColumn("group_id", F.sum("new_record").over(windowSpec))

# from pyspark.ml.feature import StringIndexerModel
# inputs1 = ["job_id"]
# outputs1 = ["job_code"]
# if os.path.exists(f'{file_path}/sparksteps/models/StringIndexer_prepare'):
#     model1 = StringIndexerModel.load(f'{file_path}/sparksteps/models/StringIndexer_prepare')
# log_test = model1.transform(log_test)

log_test = log_test.groupBy("geek_id", "group_id").agg(
    F.collect_list("job_id").alias("job_id_list"),
    F.collect_list("deal_type").alias("deal_type_list"),
    F.collect_list("deal_time").alias("deal_time_list"),
    F.collect_list("date").alias("date_list"),
    F.collect_list("pred_person").alias("pred_person_list"),
    F.collect_list("pred_job").alias("pred_job_list")
).drop("group_id").withColumn("list_size", F.size("job_id_list"))
log_test.show()


# For training data
log_train.write.mode('overwrite').parquet(f'file:////root/target_dir/JobRec-main/sparksteps/train/log_res')
print('log train cnt:', log_train.count())

# For testing data
log_test.write.mode('overwrite').parquet(f'file:////root/target_dir/JobRec-main/sparksteps/test/log_res')
print('log test cnt:', log_test.count())




dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']


+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
| geek_id|expect_id|   job_id|deal_type|page|rank| list_time| deal_time|        ds|      date|pred_person|pred_job|
+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
|12542372|  2676929|284394150|  no_list|   1|  71|1686585317|1686585317|2023-06-12|2023-06-12|         54|       0|
+--------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
only showing top 1 row

log train cnt: 1478255
+---------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
|  geek_id|expect_id|   job_id|deal_type|page|rank| list_time| deal_time|        ds|      date|pred_person|pred_job|
+---------+---------+---------+---------+----+----+----------+----------+----------+----------+-----------+--------+
|123508450|609796781|2

log train cnt: 10725
log test cnt: 4109


In [10]:
pip install pyarrow fastparquet

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [11]:
import sys
import os
import json
import time
import datetime
import pandas as pd
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

import datetime 

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 5.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"

ddays = 7
LIST_CNT = 30

K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]

def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

spark = SparkSession\
    .builder\
    .appName("spark_data_query")\
    .config("spark.sql.shuffle.partitions",500)\
    .config("spark.driver.memory", "32g")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# For training data
folder_path_train = f'{path}/sparksteps/train/log_res'
parquet_files = [os.path.join(folder_path_train, f) for f in os.listdir(folder_path_train) if f.endswith('.parquet')]
df_train = pd.concat([pd.read_parquet(file) for file in parquet_files], ignore_index=True)
# display(df_train)
print('count:', df_train.shape[0])

# For testing data
folder_path_test = f'{path}/sparksteps/test/log_res'
parquet_files = [os.path.join(folder_path_test, f) for f in os.listdir(folder_path_test) if f.endswith('.parquet')]
df_test = pd.concat([pd.read_parquet(file) for file in parquet_files], ignore_index=True)
# display(df_test)
print('count:', df_test.shape[0])

# For training data
df_train = df_train[df_train['list_size'] > LIST_CNT]
# display(df_train)
print('count:', df_train.shape[0])

# For testing data
df_test = df_test[df_test['list_size'] > LIST_CNT]
# display(df_test)
print('count:', df_test.shape[0])

# For training data
if not os.path.exists(f'{path}/sparksteps/train/relation'):
    os.mkdir(f'{path}/sparksteps/train/relation')
cnt_dict_train = {i:0 for i in range(K_person)}
for index, row in df_train.iterrows():
#     print(index, row)
    pred_job_list = row[6]
    pred_person = row[5][0]
    job_id_list = row[1]
    geek_id = row[0]
    date_list = row[4]
    unixtime_list = row[3]
    cnt = cnt_dict_train[pred_person]
#     print(str(pred_person) + ':'+str(cnt))
    relation_file = open(f'{path}/sparksteps/train/relation/relation_'+str(pred_person)+'_'+str(cnt)+'.csv', 'w')
    relation_file.write('id,pred_job,unix_time,date,job_id,geek_id\n')
    relation_file.close()
    relation_file = open(f'{path}/sparksteps/train/relation/relation_'+str(pred_person)+'_'+str(cnt)+'.csv', 'a')
    for i in range(len(pred_job_list)):
        relation_file.write(str(i)+','+str(pred_job_list[i])+','+str(unixtime_list[i])+','+str(date_list[i])+','+str(job_id_list[i])+','+str(geek_id)+'\n')
    relation_file.close()
    cnt_dict_train[pred_person] = cnt + 1
    
print(len(cnt_dict_train))

# For testing data
if not os.path.exists(f'{path}/sparksteps/test/relation'):
    os.mkdir(f'{path}/sparksteps/test/relation')
cnt_dict_test = {i:0 for i in range(K_person)}
for index, row in df_test.iterrows():
#     print(index, row)
    pred_job_list = row[6]
    pred_person = row[5][0]
    job_id_list = row[1]
    geek_id = row[0]
    date_list = row[4]
    unixtime_list = row[3]
    cnt = cnt_dict_test[pred_person]
#     print(str(pred_person) + ':'+str(cnt))
    relation_file = open(f'{path}/sparksteps/test/relation/relation_'+str(pred_person)+'_'+str(cnt)+'.csv', 'w')
    relation_file.write('id,pred_job,unix_time,date,job_id,geek_id\n')
    relation_file.close()
    relation_file = open(f'{path}/sparksteps/test/relation/relation_'+str(pred_person)+'_'+str(cnt)+'.csv', 'a')
    for i in range(len(pred_job_list)):
        relation_file.write(str(i)+','+str(pred_job_list[i])+','+str(unixtime_list[i])+','+str(date_list[i])+','+str(job_id_list[i])+','+str(geek_id)+'\n')
    relation_file.close()
    cnt_dict_test[pred_person] = cnt + 1
    
print(len(cnt_dict_test))

# For training data
new_dict = {key: value for key, value in cnt_dict_train.items() if value != 0}
print(new_dict)
print('count:', len(new_dict))

# For testing data
new_dict = {key: value for key, value in cnt_dict_test.items() if value != 0}
print(new_dict)
print('count:', len(new_dict))

import pickle as pkl

# For training data
with open(f'{path}/sparksteps/train/relation/dict.pkl', 'wb') as f:
    pkl.dump(cnt_dict_train, f)

# For testing data
with open(f'{path}/sparksteps/test/relation/dict.pkl', 'wb') as f:
    pkl.dump(cnt_dict_test, f)



dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']
count: 10725
count: 4109
count: 4670
count: 1778


/tmp/ipykernel_580151/1915265353.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_job_list = row[6]
/tmp/ipykernel_580151/1915265353.py:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_person = row[5][0]
/tmp/ipykernel_580151/1915265353.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  job_id_list = row[1]
/tmp/ipykernel_580151/1915265353.py:96: FutureWarning: Series.__getitem__ treating keys as positions is deprec

1000


/tmp/ipykernel_580151/1915265353.py:118: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_job_list = row[6]
/tmp/ipykernel_580151/1915265353.py:119: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_person = row[5][0]
/tmp/ipykernel_580151/1915265353.py:120: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  job_id_list = row[1]
/tmp/ipykernel_580151/1915265353.py:121: FutureWarning: Series.__getitem__ treating keys as positions is de

1000
{0: 1, 1: 1, 2: 1, 3: 1, 4: 7, 5: 1, 6: 11, 7: 4, 8: 4, 9: 8, 10: 5, 11: 5, 12: 4, 13: 1, 14: 1, 15: 2, 16: 12, 17: 5, 18: 2, 19: 7, 20: 6, 21: 2, 22: 5, 23: 4, 24: 2, 25: 2, 26: 7, 27: 4, 28: 4, 29: 4, 30: 2, 31: 7, 32: 6, 33: 2, 34: 12, 35: 5, 36: 2, 37: 17, 38: 9, 39: 12, 40: 16, 41: 8, 42: 1, 43: 1, 44: 1, 45: 3, 46: 2, 47: 7, 48: 4, 49: 3, 50: 1, 51: 17, 52: 2, 53: 3, 54: 5, 56: 6, 57: 7, 58: 7, 59: 6, 60: 3, 61: 7, 62: 5, 63: 10, 64: 1, 65: 1, 66: 2, 67: 1, 68: 7, 69: 1, 70: 6, 71: 2, 72: 2, 73: 3, 74: 6, 75: 9, 76: 4, 77: 3, 78: 2, 79: 2, 81: 4, 82: 7, 83: 1, 84: 14, 85: 2, 86: 4, 87: 5, 88: 5, 89: 2, 90: 2, 91: 4, 92: 4, 93: 13, 94: 4, 95: 1, 96: 4, 97: 3, 98: 1, 99: 7, 100: 1, 101: 1, 102: 11, 103: 10, 104: 3, 105: 6, 106: 2, 107: 1, 108: 18, 109: 1, 110: 1, 111: 16, 112: 5, 113: 1, 114: 8, 115: 1, 116: 1, 117: 1, 118: 6, 119: 2, 120: 2, 121: 10, 122: 7, 123: 3, 124: 4, 125: 10, 126: 1, 127: 1, 128: 10, 129: 1, 130: 12, 131: 8, 132: 5, 134: 3, 135: 3, 136: 1, 137: 2, 138:

In [12]:
import sys
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import random
import numpy as np
import pandas as pd
import datetime 

# 模拟命令行参数
sys.argv = [
     "JobRec/sparksteps/1.py",  # 脚本名称
    "2023-06-03",  # start_date
    "2023-06-30",  # end_date
    "/root/target_dir/JobRec-main",  # path
    "bj"  # city_short
]

# 以下是 6.py 脚本的其余代码
os.environ["HADOOP_USER_NAME"] = "xxx"


ddays = 7
LIST_CNT = 30
TOP_K = 20

K_job = 500
K_person = 1000  # number of person classes ### same as the K in 16.data_xxx.ipynb
numTopics = 1000  
test_days = 5
POS_CODE_LOW = '100000'
POS_CODE_HIGH = '110000'
start_date = sys.argv[1]
end_date = sys.argv[2]
path = sys.argv[3]
file_path = 'file:///' + path
city_short = sys.argv[4]


def generate_train_test_dates(start_date, end_date):
    start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    date_list_train = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range(0, (end - start).days + 1 - test_days)]
    date_list_test = [(start + datetime.timedelta(days=x)).strftime('%Y-%m-%d') 
                 for x in range((end - start).days + 1 - test_days, (end - start).days + 1)]
    
    return date_list_train, date_list_test

dates_train, dates_test = generate_train_test_dates(start_date, end_date)
print('dates tain:', dates_train)
print('dates test:', dates_test)

# For training data
dict_train = {}
with open(f'{path}/sparksteps/train/relation/dict.pkl', 'rb') as f:
    dict_train = pkl.load(f)
print('count train:', len(dict_train))

# For testing data
dict_test = {}
with open(f'{path}/sparksteps/test/relation/dict.pkl', 'rb') as f:
    dict_test = pkl.load(f)
    
from scipy.sparse import coo_matrix, linalg

def A_cal(row, col, values, N, M, deg_n, deg_e, w_e):
    H = coo_matrix((values, (row, col)), shape=(N, M)).tocsc()
    row1 = range(N)
    col1 = range(N)
    Dn = coo_matrix((deg_n, (row1, col1)), shape=(N, N)).tocsc()
    deg_n_inv = []
    for i in deg_n:
        deg_n_inv.append(1.0/(i+0.000001))
    Dn_inv = coo_matrix((deg_n_inv, (row1, col1)), shape=(N, N)).tocsc()
    row2 = range(M)
    col2 = range(M)
    De = coo_matrix((deg_e, (row2, col2)), shape=(M, M)).tocsc()
    deg_e_inv = []
    for i in deg_e:
        deg_e_inv.append(1.0/i)
    De_inv = coo_matrix((deg_e_inv, (row2, col2)), shape=(M, M)).tocsc()
    We = coo_matrix((w_e, (row2, col2)), shape=(M, M)).tocsc()
    tmp = H.dot(We)
    tmp2 = De_inv.dot(H.T)
    A = tmp.dot(tmp2)
#     print(A.shape)
    return A

# For training data
kk = 0
if not os.path.exists(f'{path}/sparksteps/train/sample'):
    os.mkdir(f'{path}/sparksteps/train/sample')
for a in range(K_person):
    b = dict_train[a]
    if a % 100 == 0:
        print('person', a, ':', b)
    if b == 0:
        continue
    
    # b!=0
    # cc = random.randint(0,b-1)
    for cc in range(b):
        
        # row: different sessions for the same person class, col: item list
        joblst = []
        label = -1

        job_relation_dict = {}
        for i in range(K_job):
            job_relation_dict[i] = []
        
        # two views of hypergraphs
        # 同一用户class，不同session，每一session构成一条超边
        H1_row = []
        H1_col = []
        H1_val = []
        De1 = []
        Dv1 = [0 for i in range(K_job)]

        # 同一类用户class，当前Item对后一所有出现的Item构建超边
        H2_row = []
        H2_col = []
        H2_val = []
        De2 = []
        Dv2 = [0 for i in range(K_job)]
    
        cnt = 0
        geek_id = -1
        for c in range(b):
            file = f'{path}/sparksteps/train/relation/relation_'+str(a)+'_'+str(c)+'.csv'
            df = pd.read_csv(file)

            job_list = df['pred_job'].values.tolist()
            if cc == c:
                geek_id = str(df['geek_id'].values[0])
                label_job = job_list[-TOP_K:] 
                job_list = job_list[:-TOP_K]
            joblst = job_list
            label = label_job

            tmp_lst = []
            tmp_cnt = 0
            last = -1

            for jobID in job_list:
                if last >= 0 and jobID not in job_relation_dict[last]:
                    job_relation_dict[last].append(jobID)
                last = jobID


                if jobID not in tmp_lst:
                    tmp_lst.append(jobID)
                    tmp_cnt += 1
                    Dv1[jobID] += 1
                    H1_row.append(jobID)
                    H1_col.append(cnt)
                    H1_val.append(1.0)
            De1.append(tmp_cnt)
            cnt += 1

        cnt = 0
        for key, vallst in job_relation_dict.items():
            for value in vallst:
                H2_row.append(value)
                H2_col.append(cnt)
                H2_val.append(1.0)
                Dv2[value] += 1
            num = len(vallst)
            if num > 0:
                De2.append(num)
                cnt += 1
        N1 = len(Dv1)
        # print('\tN1:', N1)
        M1 = len(De1)
        # print('\tM1:', M1)
        We1 = []
        for i in range(M1):
            We1.append(1.0)

        A1 = A_cal(H1_row, H1_col, H1_val, N1, M1, Dv1, De1, We1)


        N2 = len(Dv2)
        # print('\tN2:', N2)
        M2 = len(De2)
        # print('\tM2:', M2)
        We2 = []
        for i in range(M2):
            We2.append(1.0)

        A2 = A_cal(H2_row, H2_col, H2_val, N2, M2, Dv2, De2, We2)
        clustercenters = []
        with open(f'{path}/sparksteps/models/Clustercenters_occ.pkl', 'rb') as f:
            clustercenters = pkl.load(f)
        Fea = np.array(clustercenters)

        with open(f'{path}/sparksteps/train/sample/datasample_'+str(kk)+'.pkl', 'wb') as f:
            data = (A1, A2, Fea, joblst, label, geek_id)
            if a % 100 == 0:
                print('\t', geek_id)
            pkl.dump(data, f)
            kk += 1
kk_train = kk

# For testing data
kk = 0
if not os.path.exists(f'{path}/sparksteps/test/sample'):
    os.mkdir(f'{path}/sparksteps/test/sample')
for a in range(K_person):
    b = dict_test[a]
    if a % 100 == 0:
        print('person', a, ':', b)
    if b == 0:
        continue

    # cc = random.randint(0,b-1)
    for cc in range(b):
        
        # row: different sessions for the same person class, col: item list
        joblst = []
        label = -1

        job_relation_dict = {}
        for i in range(K_job):
            job_relation_dict[i] = []
        
        # two views of hypergraphs
        H1_row = []
        H1_col = []
        H1_val = []
        De1 = []
        Dv1 = [0 for i in range(K_job)]

        H2_row = []
        H2_col = []
        H2_val = []
        De2 = []
        Dv2 = [0 for i in range(K_job)]
    
        cnt = 0
        geek_id = -1
        for c in range(b):
            file = f'{path}/sparksteps/test/relation/relation_'+str(a)+'_'+str(c)+'.csv'
            df = pd.read_csv(file)

            job_list = df['pred_job'].values.tolist()
            if cc == c:
                geek_id = str(df['geek_id'].values[0])
                label_job = job_list[-TOP_K:] 
                job_list = job_list[:-TOP_K]
            joblst = job_list
            label = label_job

            tmp_lst = []
            tmp_cnt = 0
            last = -1

            for jobID in job_list:
                if last >= 0 and jobID not in job_relation_dict[last]:
                    job_relation_dict[last].append(jobID)
                last = jobID


                if jobID not in tmp_lst:
                    tmp_lst.append(jobID)
                    tmp_cnt += 1
                    Dv1[jobID] += 1
                    H1_row.append(jobID)
                    H1_col.append(cnt)
                    H1_val.append(1.0)
            De1.append(tmp_cnt)
            cnt += 1

        cnt = 0
        for key, vallst in job_relation_dict.items():
            for value in vallst:
                H2_row.append(value)
                H2_col.append(cnt)
                H2_val.append(1.0)
                Dv2[value] += 1
            num = len(vallst)
            if num > 0:
                De2.append(num)
                cnt += 1


        N1 = len(Dv1)
        # print('\tN1:', N1)
        M1 = len(De1)
        # print('\tM1:', M1)
        We1 = []
        for i in range(M1):
            We1.append(1.0)

        A1 = A_cal(H1_row, H1_col, H1_val, N1, M1, Dv1, De1, We1)
        # print('\tA1 shape:', A1.shape)


        N2 = len(Dv2)
        # print('\tN2:', N2)
        M2 = len(De2)
        # print('\tM2:', M2)
        We2 = []
        for i in range(M2):
            We2.append(1.0)

        A2 = A_cal(H2_row, H2_col, H2_val, N2, M2, Dv2, De2, We2)    
        # print('\tA2 shape:', A2.shape)

        # print('\tjob list:', joblst)
        # print('\tjob list len:', len(joblst))
        # print('\tnext job ID:', label)

        clustercenters = []
        with open(f'{path}/sparksteps/models/Clustercenters_occ.pkl', 'rb') as f:
            clustercenters = pkl.load(f)
        Fea = np.array(clustercenters)
        # print('\tocc features shape:', Fea.shape)

        with open(f'{path}/sparksteps/test/sample/datasample_'+str(kk)+'.pkl', 'wb') as f:
            data = (A1, A2, Fea, joblst, label, geek_id)
            if a % 100 == 0:
                print('\t', geek_id, ',', joblst, ',', label)
            pkl.dump(data, f)
            kk += 1
kk_test = kk

print('total train sample count:', kk_train)
print('total test sample count:', kk_test)

dates tain: ['2023-06-03', '2023-06-04', '2023-06-05', '2023-06-06', '2023-06-07', '2023-06-08', '2023-06-09', '2023-06-10', '2023-06-11', '2023-06-12', '2023-06-13', '2023-06-14', '2023-06-15', '2023-06-16', '2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20', '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24', '2023-06-25']
dates test: ['2023-06-26', '2023-06-27', '2023-06-28', '2023-06-29', '2023-06-30']
count train: 1000
person 0 : 1
	 34706274
person 100 : 1
	 529034633
person 200 : 1
	 547096422
person 300 : 2
	 38100022
	 29809914
person 400 : 3
	 103725393
	 82804470
	 550102510
person 500 : 3
	 72193900
	 32261401
	 589703810
person 600 : 0
person 700 : 3
	 45410519
	 3025525
	 77082294
person 800 : 4
	 54038196
	 38791694
	 504576735
	 39240471
person 900 : 4
	 80525208
	 67749150
	 35366431
	 45628204
person 0 : 1
	 39472752 , [2, 2, 2, 2, 2, 37, 87, 291, 451, 451, 320, 0, 1, 2, 2, 2, 2, 5, 87, 138, 320, 320, 338, 0, 2, 451, 0, 2, 2, 2, 78, 87, 138, 451, 0, 1, 0, 1, 2